In [ ]:
# default_exp funcs.merfish_comparison

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from decode_fish.imports import *

# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from decode_fish.funcs.dataset import *
from decode_fish.funcs.plotting import *
from decode_fish.engine.noise import estimate_noise_scale
import shutil
from decode_fish.funcs.visualization import *
from decode_fish.funcs.predict import predict
import torch.nn as nn
import torch.nn.functional as F
import torch.tensor as T
from decode_fish.funcs.predict import *

from omegaconf import open_dict
from hydra import compose, initialize
from decode_fish.funcs.merfish_eval import *

sys.path.append('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/FQ/istdeco/')

from istdeco import ISTDeco
from utils import random_codebook, random_image_stack
from codebook import Codebook
from starfish.image import Filter

sys.path.append('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/FQ/bardensr/')
import bardensr

import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

2021-10-01 03:24:36.308642: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
# cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_ci_13/start_mic:5000xpsf_noise:0.4/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sl_9a/old_new_format/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sl_11/seed:1xint_fac:0.1/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_1/old_loss:Truexmin_int_sig:1.0/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_1/old_loss:False/train.yaml')
# csv_name = 'test'
# cfg.foci.n_foci_avg = 0

In [ ]:
# cfg.exp_type.pred_z=False
# cfg.PSF.psf_extent_zyx=[1,21,21]
# cfg.run_name = 'test'

In [ ]:
from decode_fish.funcs.merfish_eval import *
bench_df, code_ref, targets = get_benchmark()
code_inds = np.stack([np.nonzero(c)[0] for c in code_ref])

13832


In [ ]:
# cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_2/test1/train.yaml')
# cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_1b/old_loss:Falsexpsf_noise:0.0xint_fac:0.0/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_cinfsim_1b/step_size:10000///train.yaml')

In [ ]:
# cfg = OmegaConf.load('../config/experiment/MERFISH_ci_2d_sim1.yaml')
# cfg.output.save_dir = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//models/fishcod/MERFISH_starfish/nb_run/test/'

In [ ]:
crop_sz = 128
cfg.foci.n_foci_avg = 0
cfg.training.bs = 5
cfg.PSF.n_cols = 1
cfg.random_crop.crop_sz = crop_sz
cfg.data_path.image_shape = [16,1,crop_sz,crop_sz]
cfg.model.n_p_layers = 1

model, post_proc, micro, img_3d, decode_dl = load_all(cfg)
psf, noise, micro = load_psf_noise_micro(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 128, 128)
1 volumes


In [ ]:
from decode_fish.engine.point_process import *
from decode_fish.funcs.output_trafo import *
from decode_fish.funcs.evaluation import *
import h5py
from decode_fish.funcs.routines import *

with torch.no_grad():

    x, local_rate, background = next(iter(decode_dl))
    sim_vars = PointProcessUniform(local_rate*0 + 0.1, model.int_dist.int_conc, model.int_dist.int_rate, 
                                   model.int_dist.int_loc, channels=16, n_bits=4, sim_z=False, codebook=torch.tensor(code_inds), int_option=3).sample(from_code_book=True, phasing=cfg.exp_type.phasing)
    xsim = micro(*sim_vars[:-1], add_noise=False)
    xsimn = micro.noise(xsim, background).sample()

    gt_vars = sim_vars[:-1]
    gt_df = px_to_nm(sample_to_df(*sim_vars[:-2], codes=sim_vars[-1], px_size_zyx=[1.,1.,1.]))

# with h5py.File('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_data1/dens_0.1_int_1.0.h5', 'r') as f:

#     xsimn = f['frames'][()]
#     gt_df = get_df_from_hdf5(f['locations']) 
        
# with h5py.File('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_data1/sim_2/dens_0.1_int_1.0.h5', 'r') as f:

#     dec_df = get_df_from_hdf5(f['old_loss:False']['locations_int'])

In [ ]:
# gt_df = get_code_from_ints(gt_df, code_ref, targets, int_str='', p_str='')
# gt_df = exclude_borders(gt_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

In [ ]:
# gt_df = gt_df.loc[:,['frame_idx', 'code_inds', 'x', 'y', 'z']+[f'int_{i}' for i in range(16)]]
# np.save('../data/chen_bardensr/sim_data.npy',cpu(xsimn))
# gt_df.to_csv('../data/chen_bardensr/gt_df.csv', index=False)
# np.save('../data/chen_bardensr/codebook.npy',cpu(code_ref))

In [ ]:
model, post_proc, micro, img_3d, decode_dl = load_all(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 128, 128)
1 volumes


In [ ]:
with torch.no_grad():
    
    model.eval().cuda()
    res_dict = model(T(xsimn).cuda())
    res_dict = model.tensor_to_dict(res_dict)
    pred_df = post_proc.get_df(res_dict, softmax=False)
#     pred_df = get_code_from_ints(pred_df, code_ref, targets, func=vcorrcoef, int_str='', p_str='')
#     pred_df = exclude_borders(pred_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [ ]:
# for c in range(30):
# #     c = 72

#     sub_gt = gt_df[gt_df['frame_idx'] == 1]
#     sub_gt = sub_gt[sub_gt['code_inds'] == c]
    
#     sub_pred = pred_df[pred_df['frame_idx'] == 1]
#     sub_pred = sub_pred[sub_pred['code_inds'] == c]

#     plt.figure(figsize=(10,10))
#     im = plt.imshow(cpu(torch.sigmoid(res_dict['logits']))[1,c,0])
#     add_colorbar(im)
#     plt.scatter(sub_gt['x']/100, sub_gt['y']/100, ec='red', s=100, marker='+')
#     plt.scatter(sub_pred['x']/100, sub_pred['y']/100, ec='green', fc='none', s=100, marker='o', linewidth=3)
#     plt.show()

In [ ]:
sub_df = percentile_filter(pred_df, 100, 'comb_sig')
perf, matches, _  = matching(gt_df, sub_df, match_genes=False, tolerance=300)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, sub_df, match_genes=True, tolerance=300)

Recall: 0.846
Precision: 0.958
Jaccard: 81.504
RMSE_vol: 39.207
Eff_3d: 73.048
FN: 1252.0 FP: 303.0
Num. matches: 6852.000
Shift: 0.29,-0.26,0.00
------- 0.9474605954465849
Recall: 0.813
Precision: 0.920
Jaccard: 75.917
RMSE_vol: 25.406
Eff_3d: 72.772
FN: 1519.0 FP: 570.0
Num. matches: 6585.000
Shift: 0.10,-0.12,0.00


In [ ]:
# export
def get_istdeco_df(sim_data, codebook, psf_sig=(1.7, 1.7), n_iter=100, bg=100.):
    
    istd_results = pd.DataFrame()
    
    n_rounds = codebook.shape[1]
    n_cols = codebook.shape[2]
    
    for i in range(len(sim_data)):

        image_data = np.array(sim_data)[i,:,0]
        image_data = image_data.reshape([n_rounds,n_cols,image_data.shape[-2],image_data.shape[-1]], order='F')
        
        init_th = np.percentile(image_data,50)

        istdeco_model = ISTDeco(image_data, codebook, psf_sig, b=bg)
        X, Q, loss = istdeco_model.run(niter=n_iter)

        # Get codes
        code_id, y, x = np.where(np.logical_and(X>init_th,Q>0.1))

        intensity = X[code_id, y, x]
        quality = Q[code_id, y, x]

        # Store in dataframe
        df = pd.DataFrame(data={
                'frame_idx': i,
                'x': x.astype('float')+0.5, 
                'y': y.astype('float')+0.5,
                'z': 0.5,
                'intensity': intensity, 
                'quality': quality,
                'code_inds': code_id
        })

        istd_results = istd_results.append(df)

    istd_results = px_to_nm(istd_results)

    return istd_results

In [ ]:
istd_df = get_istdeco_df(cpu(xsimn), code_ref.reshape([140,8,2], order='F'), psf_sig=(1.7, 1.7), n_iter=400)
# istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

In [ ]:
sub_df = istd_df[istd_df['quality'] > 0.55]
sub_df = sub_df[sub_df['intensity'] > 350]

In [ ]:
perf, matches, _  = matching(gt_df, sub_df, match_genes=False, tolerance=300)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, sub_df, match_genes=True, tolerance=300)

Recall: 0.968
Precision: 0.989
Jaccard: 95.816
RMSE_vol: 44.111
Eff_3d: 77.551
FN: 52.0 FP: 17.0
Num. matches: 1580.000
Shift: 1.46,-0.33,0.00
------- 0.9867088607594937
Recall: 0.964
Precision: 0.986
Jaccard: 95.106
RMSE_vol: 44.159
Eff_3d: 77.385
FN: 58.0 FP: 23.0
Num. matches: 1574.000
Shift: 1.32,-0.70,0.00


In [ ]:
istd_df = get_istdeco_df(cpu(xsimn), code_ref.reshape([140,8,2], order='F'), psf_sig=(1.7, 1.7), n_iter=400)
# istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

q_max = 0.3*istd_df['quality'].max()
i_max = 0.3*istd_df['intensity'].max()
i_min = istd_df['intensity'].min()

def objective(trial):
    
    n_iter = 400 # trial.suggest_uniform('n_iter', 50, 500)
    psf_s = 1.7 # trial.suggest_uniform('psf_s', 1.5, 1.7)
    
#     istd_df = get_istdeco_df(xsimn, code_ref.reshape([140,8,2], order='F'), psf_sig=(psf_s, psf_s), n_iter=int(n_iter))
#     istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
    
    qual_th = trial.suggest_uniform('qual_th', 0.1, q_max)
    int_th = trial.suggest_uniform('int_th', i_min, i_max)
    
    sub_df = istd_df[(istd_df['intensity'] > int_th) & (istd_df['quality'] > qual_th)]
    perf, matches, _  = matching(gt_df, sub_df, match_genes=True, print_res=False) 
    
    return -perf['jaccard']

study = optuna.create_study()
study.optimize(objective, n_trials=200)

print(study.best_params)
print('Jaccard ', -study.best_value)

[I 2021-10-01 03:29:20,786] A new study created in memory with name: no-name-effea13f-0cd0-46bc-813f-cc233a86e096
[I 2021-10-01 03:29:22,881] Trial 0 finished with value: -0.6513402880265281 and parameters: {'qual_th': 0.12156727598966141, 'int_th': 337.5451437088012}. Best is trial 0 with value: -0.6513402880265281.
[I 2021-10-01 03:29:22,916] Trial 1 finished with value: -0.04096742248352857 and parameters: {'qual_th': 0.7225380870449924, 'int_th': 6915.454900882743}. Best is trial 0 with value: -0.6513402880265281.
[I 2021-10-01 03:29:22,979] Trial 2 finished with value: -0.144990124753452 and parameters: {'qual_th': 0.7852841806068025, 'int_th': 3802.3849643536605}. Best is trial 0 with value: -0.6513402880265281.
[I 2021-10-01 03:29:23,099] Trial 3 finished with value: -0.2586057925081852 and parameters: {'qual_th': 0.47264094558035574, 'int_th': 2718.3528577143306}. Best is trial 0 with value: -0.6513402880265281.
[I 2021-10-01 03:29:23,197] Trial 4 finished with value: -0.211994

[I 2021-10-01 03:29:43,387] Trial 37 finished with value: -0.5925387453357862 and parameters: {'qual_th': 0.4283157978097917, 'int_th': 478.3196236145184}. Best is trial 32 with value: -0.7008870467138242.
[I 2021-10-01 03:29:43,780] Trial 38 finished with value: -0.5138175467396051 and parameters: {'qual_th': 0.31167326902164405, 'int_th': 1364.6704837336626}. Best is trial 32 with value: -0.7008870467138242.
[I 2021-10-01 03:29:43,920] Trial 39 finished with value: -0.27655112183172265 and parameters: {'qual_th': 0.3349755059219557, 'int_th': 2587.7783374803275}. Best is trial 32 with value: -0.7008870467138242.
[I 2021-10-01 03:29:44,001] Trial 40 finished with value: -0.17263079534474837 and parameters: {'qual_th': 0.6616158639059166, 'int_th': 3573.2386271977925}. Best is trial 32 with value: -0.7008870467138242.
[I 2021-10-01 03:29:45,512] Trial 41 finished with value: -0.6932777241273728 and parameters: {'qual_th': 0.1367152105872369, 'int_th': 472.93475493970493}. Best is trial

[I 2021-10-01 03:30:10,542] Trial 75 finished with value: -0.5161093357063867 and parameters: {'qual_th': 0.13731476508138302, 'int_th': 1366.1246642599767}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:11,751] Trial 76 finished with value: -0.709940234439104 and parameters: {'qual_th': 0.1795079129723993, 'int_th': 552.1922939291466}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:12,506] Trial 77 finished with value: -0.6518232666707978 and parameters: {'qual_th': 0.12973407401077458, 'int_th': 868.1597167425268}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:13,026] Trial 78 finished with value: -0.5785223435332688 and parameters: {'qual_th': 0.17418311661573002, 'int_th': 1129.1442238902914}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:14,242] Trial 79 finished with value: -0.7139441209880638 and parameters: {'qual_th': 0.23176994466670556, 'int_th': 321.1985977108675}. Best is trial 5

[I 2021-10-01 03:30:42,928] Trial 113 finished with value: -0.6813251430953751 and parameters: {'qual_th': 0.26397960510168333, 'int_th': 750.9921446004751}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:42,998] Trial 114 finished with value: -0.14091806167526993 and parameters: {'qual_th': 0.3100467249540083, 'int_th': 4005.7481998629564}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:43,204] Trial 115 finished with value: -0.3584304020004806 and parameters: {'qual_th': 0.7072977233153633, 'int_th': 445.6038734021905}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:43,799] Trial 116 finished with value: -0.6136554726852873 and parameters: {'qual_th': 0.2814641977898716, 'int_th': 991.4602510832531}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:30:44,578] Trial 117 finished with value: -0.6331042237547715 and parameters: {'qual_th': 0.3603609310121186, 'int_th': 126.91192019296278}. Best is tri

[I 2021-10-01 03:31:17,932] Trial 151 finished with value: -0.7164449022576923 and parameters: {'qual_th': 0.2050031284312384, 'int_th': 383.6048645249299}. Best is trial 54 with value: -0.7209405835073981.
[I 2021-10-01 03:31:19,178] Trial 152 finished with value: -0.718187714758558 and parameters: {'qual_th': 0.22435489299243866, 'int_th': 346.1492688892358}. Best is trial 54 with value: -0.7209405835073981.


KeyboardInterrupt: 

In [ ]:
sub_df = istd_df[(istd_df['intensity'] > study.best_params['int_th']) & (istd_df['quality'] > study.best_params['qual_th'])]
perf, matches, _  = matching(gt_df, sub_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, sub_df, match_genes=True)

Recall: 0.869
Precision: 0.968
Jaccard: 84.467
RMSE_vol: 79.542
Eff_3d: 57.303
FN: 1077.0 FP: 232.0
Num. matches: 7118.000
Shift: -0.36,-0.82,0.00
------- 0.829025007024445
Recall: 0.792
Precision: 0.883
Jaccard: 71.692
RMSE_vol: 52.328
Eff_3d: 61.453
FN: 1704.0 FP: 859.0
Num. matches: 6491.000
Shift: -0.06,-0.42,0.00


In [ ]:
# export
def get_bardensr_tensor(sim_data, codebook, bg=100., n_iter=400, l1_pen=0., psf_r=0):
    
    bard_results = pd.DataFrame()
    evd_tensors = []
    codeflat = codebook.T
    
    for i in range(len(sim_data)):
    
        image_data = sim_data[i]
        Xnorm = image_data - bg
#         Xnorm /= Xnorm.max()
        
#         Xnorm = bardensr.preprocessing.minmax(image_data - bg)

#         Xnorm = bardensr.preprocessing.minmax(image_data)
#         Xnorm = bardensr.preprocessing.background_subtraction(Xnorm,[0,10,10])
        Xnorm = bardensr.preprocessing.minmax(Xnorm)

        evidence_tensor_iterative,extra_learned_params=\
            bardensr.spot_calling.estimate_density_iterative(Xnorm.astype('float64'),codeflat,l1_penalty=l1_pen,use_tqdm_notebook=True,iterations=n_iter, estimate_codebook_gain=True, psf_radius=(psf_r, psf_r, psf_r))

#         evidence_tensor_iterative= bardensr.spot_calling.estimate_density_singleshot(Xnorm.astype('float64'), codeflat, noisefloor=0.05)
    
        evd_tensors.append(evidence_tensor_iterative)
        
    return evd_tensors

def get_bardensr_df(evd_tensors, th, ps=1.0):
    
    bard_results = pd.DataFrame()

    for i in range(len(evd_tensors)):    
    #     thresh_iterative=evd_tensors[i].max()*.1
        result_iterative=bardensr.spot_calling.find_peaks(evd_tensors[i],th, poolsize=(ps,ps,ps))

        code_inds = np.array(result_iterative.j.values, dtype=np.int16)
        df = pd.DataFrame(data={
                    'frame_idx': i,
                    'x': result_iterative.m2.values + 0.5, 
                    'y': result_iterative.m1.values + 0.5,
                    'z': 0.5,
                    'intensity': result_iterative.int.values,
                    'code_inds': code_inds
            })

        bard_results = bard_results.append(df)

    bard_results = px_to_nm(bard_results)
    return bard_results

In [ ]:
evd_tensors = get_bardensr_tensor(cpu(xsimn), code_ref, n_iter=100, l1_pen=.01, bg=100., psf_r=0)

2021-09-25 11:24:06.459223: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-25 11:24:06.460009: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-25 11:24:06.461228: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-09-25 11:24:06.461275: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-25 11:24:06.461340: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-09-25 11:24:06.461381: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2021-09-25 11

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
bard_df = get_bardensr_df(evd_tensors, th=.25, ps=1.05)
# bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

2021-09-25 11:24:10.373225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [ ]:
len(bard_df)

8159

In [ ]:
perf, matches, _  = matching(gt_df, bard_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, bard_df, match_genes=True)

Recall: 0.858
Precision: 0.850
Jaccard: 74.517
RMSE_vol: 143.815
Eff_3d: 23.711
FN: 1149.0 FP: 1223.0
Num. matches: 6936.000
Shift: -1.75,2.97,0.00
------- 0.5641580161476355
Recall: 0.623
Precision: 0.618
Jaccard: 44.984
RMSE_vol: 81.173
Eff_3d: 31.633
FN: 3045.0 FP: 3119.0
Num. matches: 5040.000
Shift: 0.37,1.18,0.00


In [ ]:
# evd_tensors = get_bardensr_tensor(xsimn, code_ref, n_iter=int(300), l1_pen=0.01)

def objective(trial):
    
#     n_iter = 300
#     l1_pen = trial.suggest_uniform('l1_pen', 0., 0.1)
    th     = trial.suggest_uniform('th', 0.2, 0.6)
    ps     = trial.suggest_uniform('ps', 0.1, 1.5)
    
#     evd_tensors = get_bardensr_tensor(xsimn, code_ref, n_iter=int(n_iter), l1_pen=l1_pen)
#     print(evd_tensors[0].mean())
    bard_df = get_bardensr_df(evd_tensors, th, ps)
    bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
    
    perf, matches, _  = matching(gt_df, bard_df, match_genes=True, print_res=False) 
    
    return -perf['jaccard']

study = optuna.create_study()
study.optimize(objective, n_trials=200)

print(study.best_params)
print('Jaccard ', -study.best_value)

In [ ]:
evd_tensors = get_bardensr_tensor(cpu(xsimn), code_ref, n_iter=100, l1_pen=.05, bg=100., psf_r=0)

In [ ]:
bard_df = get_bardensr_df(evd_tensors, th=study.best_params['th'], ps=study.best_params['ps'])
# bard_df = get_bardensr_df(evd_tensors, th=0.5, ps=1.05)
bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
perf, matches, _  = matching(gt_df, bard_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, bard_df, match_genes=True)

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted 19_MERFISH_routines.ipynb.
Converted 20_MERFISH_visualization.ipynb.
Converted 22_MERFISH_codenet.ipynb.
Converted 23_MERFISH_comparison.ipynb.
Converted index.ipynb.
